
### Pre Requistes for the module 

* install the `census` module before getting started. To do this, run the following command from the command line: 
    * **`pip install census`**
    
* install the `states`  module before getting started. To do this, run the following command from the command line: 
   * **`pip install states`**

* update the config file (located in src_files folder) with api key


### Documentation
* [Documentation for the profile tables in ACS1 for i year surveys] https://api.census.gov/data/2018/acs/acs1/profile
* [Census API Docs](https://www.census.gov/data/developers/data-sets.html)

##  import dependencies 





In [1]:
import pandas as pd
from census import Census #<-- Python wrapper for census API
from us import states
import requests
import os
from pprint import pprint


## create session with Census API

* import api key value 
* Create a session for Census api

In [2]:

# Census API Key
from config import api_key

# provide the api key to establish a session 
c = Census(api_key)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

## retrieve the housing statistics for Texas from Census API


In [3]:
 
# columns required from the housing api
# the variables for housing profile table can be viewed in this link :-https://api.census.gov/data/2018/acs/acs1/profile/variables.json
housingAPIColumnName = ['NAME','DP04_0001E','DP04_0002E','DP04_0003E','DP04_0016E','DP04_0017E','DP04_0018E','DP04_0019E','DP04_0020E','DP04_0021E',
                        'DP04_0022E','DP04_0023E','DP04_0024E','DP04_0025E','DP04_0049E','DP04_0050E','DP04_0051E','DP04_0052E',
                        'DP04_0053E','DP04_0054E','DP04_0055E','DP04_0079E','DP04_0080E','DP04_0081E','DP04_0082E','DP04_0083E',
                        'DP04_0084E','DP04_0085E','DP04_0086E','DP04_0087E','DP04_0088E']

#output table column Name
tableColumnName = ['Geographic Area Name', 'Estimate HOUSING OCCUPANCY Total housing units','Estimate HOUSING OCCUPANCY Occupied housing units',
                   'Estimate HOUSING OCCUPANCY Vacant housing units','Estimate YEAR STRUCTURE BUILT Total housing units','Estimate YEAR STRUCTURE BUILT Built 2005 or later',
                   'Estimate YEAR STRUCTURE BUILT Built 2000 to 2004','Estimate YEAR STRUCTURE BUILT Built 1990 to 1999','Estimate YEAR STRUCTURE BUILT Built 1980 to 1989',
                   'Estimate YEAR STRUCTURE BUILT Built 1970 to 1979','Estimate YEAR STRUCTURE BUILT Built 1960 to 1969','Estimate YEAR STRUCTURE BUILT Built 1950 to 1959',
                   'Estimate YEAR STRUCTURE BUILT Built 1940 to 1949','Estimate YEAR STRUCTURE BUILT Built 1939 or earlier','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Occupied housing units',
                   'Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 2005 or later','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 2000 to 2004','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1990 to 1999',
                   'Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1980 to 1989','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1970 to 1979','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1969 or earlier',
                   'Estimate VALUE Owner-occupied units','Estimate VALUE Less than $50,000','Estimate VALUE $50,000 to $99,999','Estimate VALUE $100,000 to $149,999',
                   'Estimate VALUE $150,000 to $199,999','Estimate VALUE $200,000 to $299,999','Estimate VALUE $300,000 to $499,999','Estimate VALUE $500,000 to $999,999',
                   'Estimate VALUE $1,000,000 or more','Estimate VALUE Median (dollars)','State', 'Geo_ID']

#years data needs to be retrieved
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

# create the data frame for storing the api responses
housingCensusDF = pd.DataFrame()

#***********build queryParameter *************

queryParameter = ""
for key in housingAPIColumnName:
    queryParameter = queryParameter + key + ","

#remove last comma
queryParameter = queryParameter[:-1]

#***********complete build queryParameter *************



#*************** build gegraphy parameters *************

# the geography parameter for the state is (48) for texas. The dataset for the state can be viewed here :- https://api.census.gov/data/2016/acs/acs5/profile?get=NAME&for=state:*
#the geography parameter for cities is quieried by place place:* indicates all the cities for the state should be retrieved :
# the link for viewing the places  :- https://api.census.gov/data/2016/acs/acs5/profile?get=NAME&for=place:*&in=state:48

geoTexasCityParameter = '&for=place:*&in=state:48'
geoUSParameter = '&for=us:*'

#*************** complete build geogrpahy parameters *******************



# ************** iterate over the years and retrieve the information from the American Housing Survery API *****************
for year in years:
    
    try : 
        
        #build url to pull national averages
        usaUrl = f"https://api.census.gov/data/{year}/acs/acs1/profile?get={queryParameter}{geoUSParameter}"
        
        try :
            
            # call the api and convert the response to json
            response = requests.get(usaUrl)
            data = response.json()
            
        except : #for query error
            
            print(f"Error While parsing the US data for the year {year}")    
            
        
        # create a new dataframe with the response values
        df = pd.DataFrame(data[1:])
               
        # assign default code for usa for future merge
        df[32] = '0001'
       

    except : # for dataframe conversion
            print(f"Error While converting response data for the frame {year}")    
      
       
             
    try : 
        
        # build url to pull city values in texas. 
        placeUrl = f"https://api.census.gov/data/{year}/acs/acs1/profile?get={queryParameter}{geoTexasCityParameter}"
        
        
        try :
            
            # call the api and convert the response to json
            response = requests.get(placeUrl).json()
            
        except :
            print(f"Error While parsing the US data for the year {year}")    
       
     
        # add the response values to the new data frame
        df = df.append(response[1:])
       
       
    except : # for dataframe conversion
            print(f"Error While converting response data for the frame {year}")    
        
    # update year
    df['Year'] = year
        
    # add the data frame with housing api responses to the main DataFrame
    housingCensusDF = housingCensusDF.append(df)

        
        
#*************** completed retrieving data from the housing profile table in the census api *******************


# rename the columns and reorder the column values

housingCensusDF = housingCensusDF.rename(columns = {0 : tableColumnName[0],
                                                    1 : tableColumnName[1],
                                                    2 : tableColumnName[2],
                                                    3 : tableColumnName[3],
                                                    4 : tableColumnName[4],
                                                    5 : tableColumnName[5],
                                                    6 : tableColumnName[6],
                                                    7 : tableColumnName[7],
                                                    8 : tableColumnName[8],
                                                    9 : tableColumnName[9],
                                                    10 : tableColumnName[10],
                                                    11 : tableColumnName[11],
                                                    12 : tableColumnName[12],
                                                    13 : tableColumnName[13],
                                                    14 : tableColumnName[14],
                                                    15 : tableColumnName[15],
                                                    16 : tableColumnName[16],   
                                                    17 : tableColumnName[17],
                                                    18 : tableColumnName[18],
                                                    19 : tableColumnName[19],
                                                    20 : tableColumnName[20],
                                                    21 : tableColumnName[21],
                                                    22 : tableColumnName[22],
                                                    23 : tableColumnName[23],
                                                    24 : tableColumnName[24],
                                                    25 : tableColumnName[25],
                                                    26 : tableColumnName[26],
                                                    27 : tableColumnName[27],
                                                    28 : tableColumnName[28],
                                                    29 : tableColumnName[29],
                                                    30 : tableColumnName[30],
                                                    31 : tableColumnName[31],
                                                    32 : tableColumnName[32] 
                                                   }) 

                                                    

newTableColumnName = ['Geo_ID', 'Geographic Area Name', 'Year', 'Estimate HOUSING OCCUPANCY Total housing units','Estimate HOUSING OCCUPANCY Occupied housing units',
                      'Estimate HOUSING OCCUPANCY Vacant housing units','Estimate YEAR STRUCTURE BUILT Total housing units','Estimate YEAR STRUCTURE BUILT Built 2005 or later',
                      'Estimate YEAR STRUCTURE BUILT Built 2000 to 2004','Estimate YEAR STRUCTURE BUILT Built 1990 to 1999','Estimate YEAR STRUCTURE BUILT Built 1980 to 1989',
                      'Estimate YEAR STRUCTURE BUILT Built 1970 to 1979','Estimate YEAR STRUCTURE BUILT Built 1960 to 1969','Estimate YEAR STRUCTURE BUILT Built 1950 to 1959',
                      'Estimate YEAR STRUCTURE BUILT Built 1940 to 1949','Estimate YEAR STRUCTURE BUILT Built 1939 or earlier','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Occupied housing units',
                      'Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 2005 or later','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 2000 to 2004','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1990 to 1999',
                      'Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1980 to 1989','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1970 to 1979','Estimate YEAR HOUSEHOLDER MOVED INTO UNIT Moved in 1969 or earlier',
                      'Estimate VALUE Owner-occupied units','Estimate VALUE Less than $50,000','Estimate VALUE $50,000 to $99,999','Estimate VALUE $100,000 to $149,999',
                      'Estimate VALUE $150,000 to $199,999','Estimate VALUE $200,000 to $299,999','Estimate VALUE $300,000 to $499,999','Estimate VALUE $500,000 to $999,999',
                      'Estimate VALUE $1,000,000 or more','Estimate VALUE Median (dollars)','State']

housingCensusDF = housingCensusDF.reindex(columns=newTableColumnName)

# update the column values for state from 48 to Texas   
housingCensusDF['State'] = 'Texas'


print('Task Completed')

Error While parsing the US data for the year 2019
Error While parsing the US data for the year 2019
Error While converting response data for the frame 2019
Task Completed


## save the output to csv file    
    

In [4]:
# Specify the file to write to
output_path = os.path.join("..", "Resources", "housingCensus.csv")

#save the file
housingCensusDF.to_csv(output_path,  index=False)
 